https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [2]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [3]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [79]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 0
%matplotlib inline
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from datetime import datetime

from helpers.feature_engineering import dateStrToDayYear, getMappingOfSimilarCategoricalColumns, \
    compareSimilarCategoricalColumns 
from helpers.my_one_hot_encoder import MyOneHotEncoder
from helpers.py_helpers import is_number
from scipy.stats import skew, kurtosis
from helpers.outliers import MyOutliers

In [5]:
rng = np.random.RandomState(seed=random_state)

# Flying to New York City - ???

In [27]:
target_col = 'IS_DELAYED'

In [6]:
path_data = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'binary_train_data_numerical.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/binary_train_data_numerical.csv'

In [7]:
df = pd.read_csv(path_data, delimiter = ',', header=0)
df.shape

(186308, 533)

In [8]:
df.sample(10, random_state=random_state)

QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK  FL_NUM  DEP_TIME  \
163263        3      9            27            2     199    1622.0   
132397        4     12             4            7     416      18.0   
15778         2      6            20            1    5443    1348.0   
156958        4     12             2            5      71    2006.0   
168959        4     11            30            3    1515    1949.0   
148799        2      5             2            1    2066    1357.0   
152433        2      4             8            5    1310    2152.0   
146704        2      6            10            5     183    1150.0   
118445        3      7            25            1      51    1013.0   
183495        2      6            28            2    5210     655.0   

        DEP_DELAY  DEP_DELAY_GROUP  DISTANCE  DISTANCE_GROUP       ...         \
163263       52.0              3.0     187.0               1       ...          
132397       88.0              5.0    1176.0               5       ...          
15778        59.0              3.0     292.0               2       ...          
156958       28.0              1.0    1990.0               8       ...          
168959       42.0              2.0     853.0               4       ...          
148799       45.0              3.0     544.0               3       ...          
152433       12.0              0.0     717.0               3       ...          
146704      -10.0             -1.0     944.0               4       ...          
118445       13.0              0.0    4983.0              11       ...          
183495       -5.0             -1.0     617.0               3       ...          

       DEP_TIME_BLK_8  DEP_TIME_BLK_9  DEP_TIME_BLK_10  DEP_TIME_BLK_11  \
163263            0.0             1.0              0.0              0.0   
132397            0.0             0.0              0.0              0.0   
15778             0.0             0.0              0.0              0.0   
156958            0.0             0.0              0.0              0.0   
168959            0.0             0.0              0.0              0.0   
148799            0.0             0.0              0.0              0.0   
152433            0.0             0.0              0.0              0.0   
146704            0.0             0.0              0.0              0.0   
118445            0.0             0.0              0.0              0.0   
183495            0.0             0.0              0.0              0.0   

        DEP_TIME_BLK_12  DEP_TIME_BLK_13  DEP_TIME_BLK_14  DEP_TIME_BLK_15  \
163263              0.0              0.0              0.0              0.0   
132397              0.0              0.0              0.0              0.0   
15778               0.0              0.0              0.0              0.0   
156958              0.0              1.0              0.0              0.0   
168959              0.0              1.0              0.0              0.0   
148799              0.0              0.0              0.0              0.0   
152433              0.0              0.0              0.0              1.0   
146704              0.0              0.0              0.0              0.0   
118445              0.0              0.0              0.0              0.0   
183495              0.0              0.0              0.0              0.0   

        DEP_TIME_BLK_16  DEP_TIME_BLK_17  
163263              0.0              0.0  
132397              1.0              0.0  
15778               0.0              0.0  
156958              0.0              0.0  
168959              0.0              0.0  
148799              0.0              0.0  
152433              0.0              0.0  
146704              0.0              0.0  
118445              0.0              0.0  
183495              0.0              0.0  

[10 rows x 533 columns]

In [10]:
df.info()  #here we are verifying that everything is numerical

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186308 entries, 0 to 186307
Columns: 533 entries, QUARTER to DEP_TIME_BLK_17
dtypes: bool(1), float64(525), int64(7)
memory usage: 756.4 MB


In [16]:
is_number("0")

True

In [32]:
columns_one_hot_encoded = np.array([col for col in df.columns if is_number(col[-1])])
columns_one_hot_encoded

array(['UNIQUE_CARRIER_0', 'UNIQUE_CARRIER_1', 'UNIQUE_CARRIER_2',
       'UNIQUE_CARRIER_3', 'UNIQUE_CARRIER_4', 'UNIQUE_CARRIER_5',
       'UNIQUE_CARRIER_6', 'UNIQUE_CARRIER_7', 'UNIQUE_CARRIER_8',
       'UNIQUE_CARRIER_9', 'UNIQUE_CARRIER_10', 'ORIGIN_0', 'ORIGIN_1',
       'ORIGIN_2', 'ORIGIN_3', 'ORIGIN_4', 'ORIGIN_5', 'ORIGIN_6',
       'ORIGIN_7', 'ORIGIN_8', 'ORIGIN_9', 'ORIGIN_10', 'ORIGIN_11',
       'ORIGIN_12', 'ORIGIN_13', 'ORIGIN_14', 'ORIGIN_15', 'ORIGIN_16',
       'ORIGIN_17', 'ORIGIN_18', 'ORIGIN_19', 'ORIGIN_20', 'ORIGIN_21',
       'ORIGIN_22', 'ORIGIN_23', 'ORIGIN_24', 'ORIGIN_25', 'ORIGIN_26',
       'ORIGIN_27', 'ORIGIN_28', 'ORIGIN_29', 'ORIGIN_30', 'ORIGIN_31',
       'ORIGIN_32', 'ORIGIN_33', 'ORIGIN_34', 'ORIGIN_35', 'ORIGIN_36',
       'ORIGIN_37', 'ORIGIN_38', 'ORIGIN_39', 'ORIGIN_40', 'ORIGIN_41',
       'ORIGIN_42', 'ORIGIN_43', 'ORIGIN_44', 'ORIGIN_45', 'ORIGIN_46',
       'ORIGIN_47', 'ORIGIN_48', 'ORIGIN_49', 'ORIGIN_50', 'ORIGIN_51',
       'ORIGIN_

In [33]:
columns_not_one_hot = np.array([col for col in df.columns if not is_number(col[-1])])
columns_not_one_hot

array(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_NUM',
       'DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_GROUP', 'DISTANCE',
       'DISTANCE_GROUP', 'IS_DELAYED', 'YDAY'], 
      dtype='|S15')

### Note for one hot encoded values
We are not exploring the skewness and kurtosis, or do any other exploration, of the one hot encoded values since we would not like to change these values at least as a first approach. These are coming from categories and it is important for now all categories to be represented with the "1" every time as all are being treated equally.
So we are not going to transform them.

In [34]:
df[columns_not_one_hot].describe()

QUARTER          MONTH   DAY_OF_MONTH    DAY_OF_WEEK  \
count  186308.000000  186308.000000  186308.000000  186308.000000   
mean        2.487215       6.472454      15.772973       3.905136   
std         1.097443       3.370194       8.794713       1.969903   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       4.000000       8.000000       2.000000   
50%         3.000000       7.000000      16.000000       4.000000   
75%         3.000000       9.000000      23.000000       5.000000   
max         4.000000      12.000000      31.000000       7.000000   

              FL_NUM       DEP_TIME      DEP_DELAY  DEP_DELAY_GROUP  \
count  186308.000000  186308.000000  186308.000000    186308.000000   
mean     1659.059954    1408.828258      31.136382         1.330437   
std      1539.062799     525.459738      67.484444         3.261172   
min         1.000000       1.000000     -31.000000        -2.000000   
25%       455.000000    1000.000000      -4.000000        -1.000000   
50%      1266.000000    1444.000000       4.000000         0.000000   
75%      2172.000000    1840.000000      42.000000         2.000000   
max      7439.000000    2400.000000    1435.000000        12.000000   

            DISTANCE  DISTANCE_GROUP           YDAY  
count  186308.000000   186308.000000  186308.000000  
mean     1057.860779        4.708145     182.244262  
std       714.357618        2.767207     103.442565  
min        94.000000        1.000000       1.000000  
25%       544.000000        3.000000      92.000000  
50%       950.000000        4.000000     183.000000  
75%      1381.000000        6.000000     269.000000  
max      4983.000000       11.000000     366.000000

In [35]:
Xtrain = df.drop(labels=[target_col], axis=1)

In [36]:
ytrain = df[target_col]

In [38]:
columns_to_explore = columns_not_one_hot[columns_not_one_hot != target_col]
#assert (len(columns_to_explore) + 1) == len(columns_not_one_hot)
len(columns_to_explore)

11